### 201600974 무역학과 김효훈 lab8

In [22]:
import numpy as np
import math

def sigmoid(v):
    return [(1/(1+(math.exp(-v_i)))) for v_i in v]


In [23]:
# 문제 1: 순전파 계산


def FowardPropagation(Input,hidden): #인풋을 가중치와 점곱해준 후 활성화함수 적용
    Xhidden = np.dot(hidden,Input)
    Ohidden = sigmoid(Xhidden)
    return np.array(Ohidden)

Input = np.array([[0.9],[0.1],[0.8]]) #입력값
Winput_hidden = np.array([[0.9,0.3,0.4],[0.2,0.8,0.2],[0.1,0.5,0.6]]) # 입력 가중치
Woutput_hidden =np.array([[0.3,0.7,0.5],[0.6,0.5,0.2],[0.8,0.1,0.9]]) # 출력 가중치

SecondInput = FowardPropagation(Input,Winput_hidden)
output=FowardPropagation(SecondInput,Woutput_hidden)

print("순전파 결과 값",output)


#나온 값을 토대로 오차를 구해줌


순전파 결과 값 [0.72630335 0.70859807 0.77809706]


In [24]:
# 문제 2: 역전파 계산

def Backpropagation(weight,Error): 
    weighted_Error = np.copy(weight)
    for i in range(len(weighted_Error)):    # 가중치의 행 수 만큼, 분모를 알맞게 나누는 과정
        weighted_Error[i] = np.divide(weighted_Error[i],sum(weighted_Error[i]))   #노드의 가중치를 더한값으로 나눔
    weighted_Error = np.transpose(weighted_Error) #나눈다음에 전치행렬을 해줌, 그래야 위 계산이 가능
    return np.dot(weighted_Error,Error)
    
Eoutput = np.array([[0.8],[0.5]]) # 책에서 오차가 이미 설정됨 
Winput_hidden =np.array([[3.0,2.0],[1.0,7.0]])   #입력층 가중치
Woutput_hidden = np.array([[2.0,3.0],[1.0,4.0]]) #출력층 가중치

Ehidden = Backpropagation(Woutput_hidden,Eoutput)
Einput = Backpropagation(Winput_hidden,Ehidden)


print("출력층의 오차\n",Eoutput)
print("은닉층의 오차\n",Ehidden)
print("입력층의 오차\n",Einput)

출력층의 오차
 [[0.8]
 [0.5]]
은닉층의 오차
 [[0.42]
 [0.88]]
입력층의 오차
 [[0.362]
 [0.938]]


In [25]:
# 문제 3: 가중치 갱신
L =0.1
def differential_sigmoid(v): #시그모이드 함수의 미분
    first = sigmoid(v)
    second = np.subtract(1,sigmoid(v))
    return np.array([q1*q2 for q1,q2 in zip(first,second)])

def update_Weight(errorOutput,outputs,weight): #가중치를 업데이트하는 함수
    innerSum = np.dot(weight,outputs) #가중치와 결과 값을 점곱하면 시그모이드함수내의 합을 구함
    innerSum=innerSum.repeat(len(innerSum)) # 시그모이드 내부 값 알맞은 값이 w11, w21 ,w12, w22 순으로 적용되게 배열
    
    errorOutput = errorOutput.repeat(len(errorOutput)) #오차값으로, 위와 같이 배열 반복으로 배열
    output = outputs                                  #output은 j에 영향을 받아서
    for i in range(len(outputs)-1):                  #번갈아가면서 수가 나와야하므로 
        output = np.append(output,outputs)            #이런식으로 반복시켜서 배열시켜줌
    
    dif_sig = differential_sigmoid(innerSum) # 그 결과를 시그모이드의 미분값에 넣음
    firstToSecond = [np.multiply(v1,v2) for v1,v2 in zip(errorOutput,dif_sig)] #첫번째두번째항 곱
    secondToThird = [np.multiply(v1,v2) for v1,v2 in zip(firstToSecond,output)] #두번째세번째항 곱
    return np.negative(np.reshape(secondToThird,(len(weight),len(weight))))



Eoutput = np.array([[0.8],[0.5]]) # 설정된 오차 값 
output = np.array([[0.4],[0.5]]) # 설정된 결과 값
Woutput_hidden = np.array([[2.0,3.0],[1.0,4.0]]) #출력층 가중치

Woutput_hidden = np.subtract(Woutput_hidden,np.multiply
                             (L,update_Weight(Eoutput,output,Woutput_hidden))) # 오차기울기에 학습률 곱하고 음수씌우기
print("순서: w11 \t w21 \t   w12 \t\t w22 \n",Woutput_hidden)

순서: w11 	 w21 	   w12 		 w22 
 [[2.00265023 3.00331278]
 [1.0015251  4.00190637]]


In [27]:
#문제 4: 역전파 오차값과 가중치 업데이트 계산
import matplotlib.pyplot as plt

L = 0.1 #학습률

targets = np.array([[0.01],[0.01],[0.99]]) #목표값

Input = np.array([[0.9],[0.1],[0.8]]) #입력값
Winput_hidden = np.array([[0.9,0.3,0.4],[0.2,0.8,0.2],[0.1,0.5,0.6]]) # 초기입력 가중치
Woutput_hidden = np.array([[0.3,0.7,0.5],[0.6,0.5,0.2],[0.8,0.1,0.9]]) # 초기출력 가중치

def Update(Input,Winput_hidden,Woutput_hidden,reps):
    for i in range(reps):
        #순전파로 결과 얻기
        InputToHidden = FowardPropagation(Input,Winput_hidden)
        output=FowardPropagation(InputToHidden,Woutput_hidden)

        #순전파 결과 후 오차 얻기
        Eoutput = np.array([((v_i - v_j)) for v_i,v_j in zip(targets,output)]) # (목표값 - 실제값)를 오차설정
        #print(sum([vi**2 for vi in Eoutput])) #매 테스트 시 오차율 확인
    
        #역전파 시작
        Ehidden = Backpropagation(Woutput_hidden,Eoutput) # 은닉층 오차
        Einput = Backpropagation(Winput_hidden,Ehidden) # 입력층 오차

        
        #은닉층 to 결과층 가중치 업데이트
        Woutput_hidden = np.subtract(Woutput_hidden,np.multiply(L,update_Weight(Eoutput,output,Woutput_hidden))) #학습률 적용한 업데이트 된 가중치들    
        #입력층 to 은닉층 가중치 업데이트
        Winput_hidden = np.subtract(Winput_hidden,np.multiply(L,update_Weight(Ehidden,InputToHidden,Winput_hidden)))
    return output
        
output = Update(Input,Winput_hidden,Woutput_hidden,1450)        
print("최종 output 값 ",output, "\n목표 output 값  [0.01 \t    0.01\t0.99]")

#목표값과 상당히 가까운 수치까지 갈 수는 있으나, 소수점 두자리 숫자는 완벽하게 다가가기 어려움,
#소수점 첫번째 숫자는 거의 완벽하게 다가감

최종 output 값  [0.091376   0.09103959 0.91341935] 
목표 output 값  [0.01 	    0.01	0.99]


### 201600974 무역학과 김효훈